# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [ ]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-d6b7ddcd0c-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Sport Workflow Series ("Running explorative search") 

Consider the following exploratory information need:

> investigate the association football players and find the main BGPs related to this sports. Compare awards and participation in the competitions of the players

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q5`       | Human         | node      |
| `wd:Q9124`    | Eliud Kipchoge| node      |
| `wd:Q853003`| athletics at the 2008 Summer Olympics – men's 200 metres |node|





Also consider

```
wd:Q9124 ?p ?obj .
```

is the BGP to retrieve all **properties of Eliud Kipchoge**


Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.


The workload should:


1. Identify the BGP for runner

2. Identify the BGP for long distance running

3. Return the disciplines of the long distance running (e.g. marathon) (the result set must be a list of couples discipline IRI and label).

4. Consider marathons ran in Europe. Return the number of different runners who won a marathon group by their country of citizenship (the result set must be a triples of country IRI, label and #different runners).

5. Identify the BGP for Olympic Games

6. Consider only the summer olympic games of 2004-2008-2012-2016. Return the number of marathon runners who had partecipated to all these editions group by their country of citizenship (if the runner partecipated in more than one edition of the game, consider him/her only once) (the result set must be a list of country IRI, label and #marathon runners)

7. For each disciplines of the long distance running find who hold the World Record (the result set must be a list of elements with discipline IRI, label and athlete IRI and label).

## Task 1
Identify the BGP for runner

In [220]:
# retrieve all properties of Eliud Kipchoge

queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q9124 ?p ?obj.
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1000'), ('name', 'record held')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P1146'), ('name', 'World Athletics athlete ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('name', 'languages spoken, written or signed')]
[('p', 'http://www.wikidata.org/prop/direct/P1447'), ('name', 'Sports-Reference.com Olympic athlete ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]
[('p', 'http://www.wikidata.org/prop/direct/P1559'), ('name', 'name in native language')]
[('p', 'http://www.wikidata.org/prop/direct/P172'), ('name', 'ethnic group')]
[('p', 'http://www.wikidata.org/prop/direct/P1741'), ('name', 'GTAA ID')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P19'), ('name', 

In [11]:
# look at the occupation of Eliud Kipchoge
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q9124 wdt:P106 ?obj.
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q13382460'), ('name', 'marathon runner')]
[('obj', 'http://www.wikidata.org/entity/Q11513337'), ('name', 'athletics competitor')]
[('obj', 'http://www.wikidata.org/entity/Q4439155'), ('name', 'long-distance runner')]
3


Final query for this task

In [19]:
# look at the subclasses of 'long-distance runner' 
# wdt:P279 subclass
# wd:Q4439155 'long-distance runner'

queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q4439155 wdt:P279 ?obj.
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)



Results
[('obj', 'http://www.wikidata.org/entity/Q12803959'), ('name', 'runner')]
1


## Task 2
Identify the BGP for long distance running

In [ ]:
# we already obtained the 'long-distance runner' entity earlier when looking at occupations of Eliud Kipchoge, let's use it to find out the BGP for long distance running

Final query for this task

In [44]:
# look at entities related to 'long-distance runner'

queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q4439155 ?p ?obj.
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q8928097'), ('name', 'Category:Long-distance runners')]
[('obj', 'http://www.wikidata.org/entity/Q13382608'), ('name', 'cross-country skier')]
[('obj', 'http://www.wikidata.org/entity/Q28640'), ('name', 'profession')]
[('obj', 'http://www.wikidata.org/entity/Q917206'), ('name', 'long-distance running')]
[('obj', 'http://www.wikidata.org/entity/Q12803959'), ('name', 'runner')]
5


## Task 3
Return the disciplines of the long distance running (e.g. marathon) (the result set must be a list of couples discipline IRI and label).

Final query for this task

In [57]:
# look at entities that have'long-distance running' as a subclass
# wdt:P279 -- subclass
# wd:Q917206 -- 'long-distance running'

queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   ?obj wdt:P279 wd:Q917206.
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q26844379'), ('name', 'one-hour run')]
[('obj', 'http://www.wikidata.org/entity/Q2815830'), ('name', '2 miles run')]
[('obj', 'http://www.wikidata.org/entity/Q215677'), ('name', 'half marathon')]
[('obj', 'http://www.wikidata.org/entity/Q163892'), ('name', '10,000 metres')]
[('obj', 'http://www.wikidata.org/entity/Q240500'), ('name', '5000 metres')]
[('obj', 'http://www.wikidata.org/entity/Q40244'), ('name', 'marathon')]
[('obj', 'http://www.wikidata.org/entity/Q2774730'), ('name', '10K run')]
[('obj', 'http://www.wikidata.org/entity/Q500050'), ('name', 'cross country running')]
[('obj', 'http://www.wikidata.org/entity/Q26303'), ('name', 'ultramarathon')]
[('obj', 'http://www.wikidata.org/entity/Q2164200'), ('name', 'one hour run')]
[('obj', 'http://www.wikidata.org/entity/Q19767716'), ('name', '15K run')]
[('obj', 'http://www.wikidata.org/entity/Q19827858'), ('name', '20 kilometres')]
[('obj', 'http://www.wikidata.org/entity/Q2538895'),

## Task 4
Consider marathons ran in Europe. Return the number of different runners who won a marathon group by their country of citizenship (the result set must be a triples of country IRI, label and #different runners).

In [61]:
# let's have a look at entities related to marathon

queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q40244 ?p ?obj.
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q13382460'), ('name', 'marathon runner')]
[('obj', 'http://www.wikidata.org/entity/Q20078554'), ('name', 'Great Soviet Encyclopedia (1926–1947)')]
[('obj', 'http://www.wikidata.org/entity/Q2312410'), ('name', 'sports discipline')]
[('obj', 'http://www.wikidata.org/entity/Q2657718'), ('name', 'Armenian Soviet Encyclopedia')]
[('obj', 'http://www.wikidata.org/entity/Q542'), ('name', 'athletics')]
[('obj', 'http://www.wikidata.org/entity/Q878123'), ('name', 'racing')]
[('obj', 'http://www.wikidata.org/entity/Q917206'), ('name', 'long-distance running')]
[('obj', 'http://www.wikidata.org/entity/Q9124'), ('name', 'Eliud Kipchoge')]
[('obj', 'http://www.wikidata.org/entity/Q31900'), ('name', 'Battle of Marathon')]
[('obj', 'http://www.wikidata.org/entity/Q63223'), ('name', 'Dennis Kipruto Kimetto')]
[('obj', 'http://www.wikidata.org/entity/Q5866964'), ('name', 'Category:Marathons')]
[('obj', 'http://www.wikidata.org/entity/Q10088591'), ('name'

In [95]:
# let's have a look at properties related to marathon

queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q40244 ?p ?obj.
   # get the label
   ?p sc:name ?name.
}
#LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1000'), ('name', 'record held')]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('name', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2347'), ('name', 'YSO ID')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2581'

In [ ]:
# inspecting both entities and properties related to 'marathon' we can't get any useful data for our query

In [120]:
# get all the marathon runners
queryString = """
SELECT DISTINCT ?runner ?name
WHERE {
   # bind something
   ?runner wdt:P106 wd:Q13382460;
        wdt:P31 wd:Q5.
   # get the label
   ?runner sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)


Results
[('runner', 'http://www.wikidata.org/entity/Q1701191'), ('name', 'John Nada Saya')]
[('runner', 'http://www.wikidata.org/entity/Q16006623'), ('name', 'Harry Payne')]
[('runner', 'http://www.wikidata.org/entity/Q16007889'), ('name', 'John Tarrant')]
[('runner', 'http://www.wikidata.org/entity/Q16008400'), ('name', 'Waldemar Karlsson')]
[('runner', 'http://www.wikidata.org/entity/Q16011969'), ('name', 'John J. Reynolds')]
[('runner', 'http://www.wikidata.org/entity/Q1602744'), ('name', 'Hellen Mugo')]
[('runner', 'http://www.wikidata.org/entity/Q16185418'), ('name', 'Jerry Kiernan')]
[('runner', 'http://www.wikidata.org/entity/Q16186777'), ('name', 'George Malley')]
[('runner', 'http://www.wikidata.org/entity/Q16210841'), ('name', 'Elva Dryer')]
[('runner', 'http://www.wikidata.org/entity/Q16212704'), ('name', 'José Alejandro Semprún')]
10


In [84]:
# let's have a look of properties related to marathon runners
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   ?runner ?p wd:Q13382460;
        wdt:P31 wd:Q5.
   ?runner ?obj ?a.
   # get the label
   ?obj sc:name ?name.
}
#LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('obj', 'http://www.wikidata.org/prop/direct/P2162'), ('name', 'Deutsche Ultramarathon-Vereinigung ID')]
[('obj', 'http://www.wikidata.org/prop/direct/P2323'), ('name', 'Swedish Olympic Committee athlete ID')]
[('obj', 'http://www.wikidata.org/prop/direct/P3653'), ('name', 'ARRS runner ID')]
[('obj', 'http://www.wikidata.org/prop/direct/P3881'), ('name', 'National Track & Field Hall of Fame athlete ID')]
[('obj', 'http://www.wikidata.org/prop/direct/P4069'), ('name', 'Italian National Olympic Committee athlete ID')]
[('obj', 'http://www.wikidata.org/prop/direct/P4050'), ('name', 'French Olympic Committee athlete ID')]
[('obj', 'http://www.wikidata.org/prop/direct/P3767'), ('name', 'French Athletics Federation athlete ID')]
[('obj', 'http://www.wikidata.org/prop/direct/P6421'), ('name', 'infosport.ru person ID')]
[('obj', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('obj', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nat

In [ ]:
# we get the following useful results
# P1532 country for sport
# P166 award received
# [('obj', 'http://www.wikidata.org/prop/direct/P3766'), ('name', 'European Athletics ID')]

In [102]:
# analizing the wd:Q853003 athletics at the 2008 Summer Olympics – men's 200 metres we get more datas that could be useful 

queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q853003 ?p ?obj.
   # get the label
   ?obj sc:name ?name.
}
#LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q133525'), ('name', 'Beijing National Stadium')]
[('obj', 'http://www.wikidata.org/entity/Q148'), ('name', "People's Republic of China")]
[('obj', 'http://www.wikidata.org/entity/Q18536594'), ('name', 'Olympic sporting event')]
[('obj', 'http://www.wikidata.org/entity/Q40970'), ('name', 'International Olympic Committee')]
[('obj', 'http://www.wikidata.org/entity/Q542'), ('name', 'athletics')]
[('obj', 'http://www.wikidata.org/entity/Q11783626'), ('name', 'athletics meeting')]
[('obj', 'http://www.wikidata.org/entity/Q1189'), ('name', 'Usain Bolt')]
[('obj', 'http://www.wikidata.org/entity/Q1649415'), ('name', "athletics at the 2012 Summer Olympics – men's 200 metres")]
[('obj', 'http://www.wikidata.org/entity/Q2313564'), ('name', "athletics at the 2004 Summer Olympics – men's 200 metres")]
[('obj', 'http://www.wikidata.org/entity/Q55243755'), ('name', "men's 200 metres")]
[('obj', 'http://www.wikidata.org/entity/Q189941'), ('name', 'athl

In [91]:
#let's check is wdt:P166 is a correct way to retrieve winners
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q1189 wdt:P166 ?obj.
   # get the label
   ?obj sc:name ?name.
}
#LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('obj', 'http://www.wikidata.org/entity/Q1061233'), ('name', "L'Équipe Champion of Champions")]
[('obj', 'http://www.wikidata.org/entity/Q11961165'), ('name', 'Bislett medal')]
[('obj', 'http://www.wikidata.org/entity/Q1808301'), ('name', 'Laureus World Sports Award for Sportsman of the Year')]
[('obj', 'http://www.wikidata.org/entity/Q14606194'), ('name', 'Commander of the Order of Distinction')]
[('obj', 'http://www.wikidata.org/entity/Q4896696'), ('name', 'Best International Athlete ESPY Award')]
[('obj', 'http://www.wikidata.org/entity/Q14606267'), ('name', 'Member of the Order of Jamaica')]
6


In [ ]:
# wdt:P166 'award received' property is not correct, let's try different

In [93]:
# let's check if we can get a property for inspecting something else than 'marathon runner' for example let's try using the entity Q1189 that stands for Usain Bolt

queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q1189 ?p ?obj.
   # get the label
   ?p sc:name ?name.
}
#LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P1000'), ('name', 'record held')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1050'), ('name', 'medical condition')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P1146'), ('name', 'World Athletics athlete ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1207'), ('name', 'NUKAT ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), ('name', 'NNDB people ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1280'), ('name', 'CONOR.SI ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1285'), ('name', 'Munzinger Sport number')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'de

In [ ]:
# even inspecting properties related to 'Usain Bolt' there's no way to retrieve the winner for a race

In [ ]:
# let's try inspecting events instead of runners

In [97]:
# retrieve all the properties related to wd:Q853003 athletics at the 2008 Summer Olympics – men's 200 metres

queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q853003 ?p ?obj.
   # get the label
   ?p sc:name ?name.
}
#LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1132'), ('name', 'number of participants')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('name', 'followed by')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('name', 'competition class')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('name', 'location')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P585'), ('name', 'point in time')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('name', 'Freebase ID')]
[('p', 'http://www.wikidata.org/prop/direct/P664'), ('name', 'organizer')]

In [ ]:
# from the previous query we get the property P1346 which returns the winner of the event

In [122]:
# let's inspect marathon runners that are also winners

queryString = """
SELECT DISTINCT ?runner ?name
WHERE {
   # bind something
   ?runner wdt:P106 wd:Q13382460;
           wdt:P31 wd:Q5;
           wdt:P1346 ?winner.
   # get the label
   ?runner sc:name ?name.
}
#LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [ ]:
# inspecting marathon runners that are also winners doesn't give us any result

In [125]:
# let's check if winner property return any data at all 

queryString = """
SELECT DISTINCT ?runner ?name
WHERE {
   # bind something
   ?runner #wdt:P106 wd:Q13382460;
           wdt:P31 wd:Q5;
           wdt:P1346 ?winner.
   # get the label
   ?runner sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('runner', 'http://www.wikidata.org/entity/Q7067552'), ('name', 'Nse Ikpe Etim')]
[('runner', 'http://www.wikidata.org/entity/Q17624314'), ('name', 'Mayu Mukaida')]
[('runner', 'http://www.wikidata.org/entity/Q87872'), ('name', 'Hellmuth Hirth')]
[('runner', 'http://www.wikidata.org/entity/Q10556987'), ('name', 'Wendy Gebauer')]
[('runner', 'http://www.wikidata.org/entity/Q523046'), ('name', 'Torbjörn Blomdahl')]
[('runner', 'http://www.wikidata.org/entity/Q3459336'), ('name', 'Joe Taslim')]
[('runner', 'http://www.wikidata.org/entity/Q7934379'), ('name', 'Virginia Leng')]
[('runner', 'http://www.wikidata.org/entity/Q10552391'), ('name', 'Julie Ertz')]
[('runner', 'http://www.wikidata.org/entity/Q20739132'), ('name', 'Carlo Bazzi')]
[('runner', 'http://www.wikidata.org/entity/Q1362169'), ('name', 'Agnez Mo')]
10


In [ ]:
# when inspecting winners without including the 'marathon runner' property we get valid results, we still can't find a way to get a marathon runner winner

In [35]:
# let's go back to inspecting events instead of runners, let's see if there's a way to get marathon events first of all

In [136]:
# get all the marathon runners that have a 'European Athletics ID' (P3766)
queryString = """
SELECT DISTINCT ?EUrunner ?name
WHERE {
   # bind something
   ?EUrunner wdt:P106 wd:Q13382460;
           wdt:P31 wd:Q5;
           wdt:P3766 ?obj.
   # get the label
   ?EUrunner sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)


Results
[('EUrunner', 'http://www.wikidata.org/entity/Q27070232'), ('name', 'Michele Fanelli')]
[('EUrunner', 'http://www.wikidata.org/entity/Q450367'), ('name', 'Alyson Dixon')]
[('EUrunner', 'http://www.wikidata.org/entity/Q256770'), ('name', 'Ingrid Kristiansen')]
[('EUrunner', 'http://www.wikidata.org/entity/Q271607'), ('name', 'Alina Ivanova')]
[('EUrunner', 'http://www.wikidata.org/entity/Q471602'), ('name', 'Viktor Röthlin')]
[('EUrunner', 'http://www.wikidata.org/entity/Q472410'), ('name', 'Luís Novo')]
[('EUrunner', 'http://www.wikidata.org/entity/Q517647'), ('name', 'Charlotte Purdue')]
[('EUrunner', 'http://www.wikidata.org/entity/Q1067015'), ('name', 'Charlie Spedding')]
[('EUrunner', 'http://www.wikidata.org/entity/Q20895412'), ('name', 'Filomena Costa')]
[('EUrunner', 'http://www.wikidata.org/entity/Q1065207'), ('name', 'Tom Richards')]
[('EUrunner', 'http://www.wikidata.org/entity/Q920828'), ('name', 'Aleksey Sokolov')]
[('EUrunner', 'http://www.wikidata.org/entity/Q1671

In [199]:
# let's get all the marathon events

queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   ?obj wdt:P279 wd:Q40244.
   # get the label
   ?obj sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q1473381'), ('name', 'Fukuoka International Marathon')]
[('obj', 'http://www.wikidata.org/entity/Q1191380'), ('name', 'Tokyo Marathon')]
[('obj', 'http://www.wikidata.org/entity/Q253394'), ('name', 'Honolulu Marathon')]
[('obj', 'http://www.wikidata.org/entity/Q175125'), ('name', 'Tiberias Marathon')]
[('obj', 'http://www.wikidata.org/entity/Q161222'), ('name', 'Berlin Marathon')]
[('obj', 'http://www.wikidata.org/entity/Q528634'), ('name', 'Frankfurt Marathon')]
[('obj', 'http://www.wikidata.org/entity/Q1071285'), ('name', 'Paris Marathon')]
[('obj', 'http://www.wikidata.org/entity/Q1631470'), ('name', 'Houston Marathon')]
[('obj', 'http://www.wikidata.org/entity/Q679106'), ('name', 'Madrid Marathon')]
[('obj', 'http://www.wikidata.org/entity/Q807910'), ('name', 'Barcelona Marathon')]
[('obj', 'http://www.wikidata.org/entity/Q752138'), ('name', 'New York City Marathon')]
[('obj', 'http://www.wikidata.org/entity/Q1202167'), ('name', 'eki

In [8]:
# let's try to filter marathons that only took place in Europe

queryString = """
SELECT DISTINCT ?marathon ?name
WHERE {
   # bind something
   ?marathon wdt:P279 wd:Q40244;
        wdt:P3766 ?obj.
   # get the label
   ?marathon sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [7]:
# let's look if there's any useful property to filter the continent Europe analizing 'Berlin Marathon'

queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q161222 ?p ?obj.
   # get the label
   ?p sc:name ?name.
}
#LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('name', 'located in the administrative territorial entity')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P2002'), ('name', 'Twitter username')]
[('p', 'http://www.wikidata.org/prop/direct/P2031'), ('name', 'work period (start)')]
[('p', 'http://www.wikidata.org/prop/direct/P214'), ('name', 'VIAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('name', 'location')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P2922'), ('name', 'month of the year')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'htt

In [ ]:
# there's no property assigning Europe as a country but there are country and location properties that could be useful

In [6]:
queryString = """
SELECT DISTINCT ?country ?name
WHERE {
   # bind something
   #wd:Q161222 wdt:P276 ?location.
   wd:Q161222 wdt:P17 ?country.
   # get the label
   ?country sc:name ?name.
}
#LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany')]
1


In [4]:
queryString = """
SELECT DISTINCT ?location ?name
WHERE {
   # bind something
   wd:Q161222 wdt:P276 ?location.
   #wd:Q161222 wdt:P17 ?country.
   # get the label
   ?location sc:name ?name.
}
#LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('location', 'http://www.wikidata.org/entity/Q64'), ('name', 'Berlin')]
1


In [ ]:
# analizing location and country we get to the point that's better to use country rather than location 

In [137]:
# let's try to filter marathons that took place in Eurore using this method (first try with only one country)

queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   VALUES ?country {wd:Q183}
   ?obj wdt:P279 wd:Q40244;
        wdt:P17 ?country.
   
   # get the label
   ?obj sc:name ?name.
   #FILTER(?country = "Germany"^^xsd:string).
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q161222'), ('name', 'Berlin Marathon')]
[('obj', 'http://www.wikidata.org/entity/Q528634'), ('name', 'Frankfurt Marathon')]
[('obj', 'http://www.wikidata.org/entity/Q1258425'), ('name', 'Dresden Marathon')]
3


In [ ]:
# the results are all the marathons that took place in Germany, now let's get all the marathons that took place in Europe

In [152]:
# let's have a look at Germany properties trying to search something that can lead us to the 'Europe' entity

queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q183 ?p ?obj.
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1081'), ('name', 'Human Development Index')]
[('p', 'http://www.wikidata.org/prop/direct/P1082'), ('name', 'population')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1198'), ('name', 'unemployment rate')]
[('p', 'http://www.wikidata.org/prop/direct/P122'), ('name', 'basic form of government')]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('name', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1249'), ('name', 'time of earliest written record')]
[('p', 'http://www.wikidata.org/prop/direct/P1273'), ('name', 'CANTIC ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1280'), ('name', 'CONOR.SI ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1281'), ('name', 'WOEID')]
[('p', 'http://www.wikidata.org/prop/di

In [ ]:
# [('p', 'http://www.wikidata.org/prop/direct/P30'), ('name', 'continent')]

In [153]:
# let's get the Entity for Europe

queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q183 wdt:P30 ?obj.
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q46'), ('name', 'Europe')]
1


In [158]:
# now that we have the entity code for Europe let's try to get all the countries located in Europe

queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q46 ?p ?obj.
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P7607'), ('name', 'WikiTree category or space')]
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('name', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1082'), ('name', 'population')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('name', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1281'), ('name', 'WOEID')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', '

In [ ]:
# there's no property returning all countries of Europe

In [ ]:
# let's have a look at Europe's related entities

In [175]:
# Q46 -- Europe

queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q46 ?p ?obj.
   # get the label
   ?obj sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q4997598'), ('name', 'Portal:Europe')]
[('obj', 'http://www.wikidata.org/entity/Q345567'), ('name', 'Europe')]
[('obj', 'http://www.wikidata.org/entity/Q43105'), ('name', 'Mount Elbrus')]
[('obj', 'http://www.wikidata.org/entity/Q1377029'), ('name', 'European studies')]
[('obj', 'http://www.wikidata.org/entity/Q119716'), ('name', 'geography of Europe')]
[('obj', 'http://www.wikidata.org/entity/Q263686'), ('name', 'Southeast Europe')]
[('obj', 'http://www.wikidata.org/entity/Q4587662'), ('name', 'Category:Europe')]
[('obj', 'http://www.wikidata.org/entity/Q122482'), ('name', 'European flag')]
[('obj', 'http://www.wikidata.org/entity/Q19180675'), ('name', 'Small Brockhaus and Efron Encyclopedic Dictionary')]
[('obj', 'http://www.wikidata.org/entity/Q2'), ('name', 'Earth')]
[('obj', 'http://www.wikidata.org/entity/Q2226947'), ('name', 'economy of Europe')]
[('obj', 'http://www.wikidata.org/entity/Q2356448'), ('name', 'Kaliningrad Time')]
[(

In [ ]:
# we see that Europe has subclasses named :'Eastern Europe', 'Western Europe', 'Central Europe'
# let's use those entities to retrieve all countries in Europe

In [185]:
# let's have a look at properties of the subregions

queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q27468 ?p ?obj.
   # get the label
   ?p sc:name ?name.
}
#LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('name', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1566'), ('name', 'GeoNames ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P2082'), ('name', 'M.49 code')]
[('p', 'http://www.wikidata.org/prop/direct/P214'), ('name', 'VIAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2184'), ('name', 'history of topic')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2347'), ('name', 'YSO ID')]
[('p', 'http://www.wikidata.org/prop/direct/P242'), ('name', 'locator map image')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress aut

In [ ]:
# there's no way of accessing all of Europe contries and save them together, so we'll have to access separetely every European region (Eastern Europe, Western Europe and Central Europe etc..) and 
# look at entities for country in each region

In [ ]:
# Looking at countries located in Eastern Europe

queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q27468 ?p ?obj.
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q184'), ('name', 'Belarus')]
[('obj', 'http://www.wikidata.org/entity/Q212'), ('name', 'Ukraine')]
[('obj', 'http://www.wikidata.org/entity/Q213'), ('name', 'Czech Republic')]
[('obj', 'http://www.wikidata.org/entity/Q214'), ('name', 'Slovakia')]
[('obj', 'http://www.wikidata.org/entity/Q217'), ('name', 'Moldova')]
[('obj', 'http://www.wikidata.org/entity/Q218'), ('name', 'Romania')]
[('obj', 'http://www.wikidata.org/entity/Q219'), ('name', 'Bulgaria')]
[('obj', 'http://www.wikidata.org/entity/Q28'), ('name', 'Hungary')]
[('obj', 'http://www.wikidata.org/entity/Q36'), ('name', 'Poland')]
[('obj', 'http://www.wikidata.org/entity/Q46'), ('name', 'Europe')]
[('obj', 'http://www.wikidata.org/entity/Q82794'), ('name', 'geographic region')]
[('obj', 'http://www.wikidata.org/entity/Q875134'), ('name', 'European Russia')]
[('obj', 'http://www.wikidata.org/entity/Q1413113'), ('name', 'Eastern European studies')]
[('obj', 'http://www.wikidata.org/

In [172]:
# Looking at countries located in Western Europe

queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q27496 ?p ?obj.
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q160381'), ('name', 'Western world')]
[('obj', 'http://www.wikidata.org/entity/Q142'), ('name', 'France')]
[('obj', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany')]
[('obj', 'http://www.wikidata.org/entity/Q235'), ('name', 'Monaco')]
[('obj', 'http://www.wikidata.org/entity/Q31'), ('name', 'Belgium')]
[('obj', 'http://www.wikidata.org/entity/Q32'), ('name', 'Luxembourg')]
[('obj', 'http://www.wikidata.org/entity/Q39'), ('name', 'Switzerland')]
[('obj', 'http://www.wikidata.org/entity/Q40'), ('name', 'Austria')]
[('obj', 'http://www.wikidata.org/entity/Q46'), ('name', 'Europe')]
[('obj', 'http://www.wikidata.org/entity/Q55'), ('name', 'Netherlands')]
[('obj', 'http://www.wikidata.org/entity/Q82794'), ('name', 'geographic region')]
[('obj', 'http://www.wikidata.org/entity/Q64147921'), ('name', 'Portal:Western Europe')]
[('obj', 'http://www.wikidata.org/entity/Q86344423'), ('name', 'Category:Western European people')]
[('obj', '

In [174]:
# Looking at countries located in Central Europe

queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q27509 ?p ?obj.
   # get the label
   ?obj sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q142'), ('name', 'France')]
[('obj', 'http://www.wikidata.org/entity/Q145'), ('name', 'United Kingdom')]
[('obj', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany')]
[('obj', 'http://www.wikidata.org/entity/Q213'), ('name', 'Czech Republic')]
[('obj', 'http://www.wikidata.org/entity/Q214'), ('name', 'Slovakia')]
[('obj', 'http://www.wikidata.org/entity/Q27'), ('name', 'Ireland')]
[('obj', 'http://www.wikidata.org/entity/Q28'), ('name', 'Hungary')]
[('obj', 'http://www.wikidata.org/entity/Q31'), ('name', 'Belgium')]
[('obj', 'http://www.wikidata.org/entity/Q32'), ('name', 'Luxembourg')]
[('obj', 'http://www.wikidata.org/entity/Q347'), ('name', 'Liechtenstein')]
[('obj', 'http://www.wikidata.org/entity/Q35'), ('name', 'Denmark')]
[('obj', 'http://www.wikidata.org/entity/Q36'), ('name', 'Poland')]
[('obj', 'http://www.wikidata.org/entity/Q39'), ('name', 'Switzerland')]
[('obj', 'http://www.wikidata.org/entity/Q40'), ('name', 'Austr

In [178]:
# Looking at countries located in Southern Europe

queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q27449 ?p ?obj.
   # get the label
   ?obj sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q1410'), ('name', 'Gibraltar')]
[('obj', 'http://www.wikidata.org/entity/Q215'), ('name', 'Slovenia')]
[('obj', 'http://www.wikidata.org/entity/Q221'), ('name', 'North Macedonia')]
[('obj', 'http://www.wikidata.org/entity/Q222'), ('name', 'Albania')]
[('obj', 'http://www.wikidata.org/entity/Q224'), ('name', 'Croatia')]
[('obj', 'http://www.wikidata.org/entity/Q225'), ('name', 'Bosnia and Herzegovina')]
[('obj', 'http://www.wikidata.org/entity/Q228'), ('name', 'Andorra')]
[('obj', 'http://www.wikidata.org/entity/Q233'), ('name', 'Malta')]
[('obj', 'http://www.wikidata.org/entity/Q236'), ('name', 'Montenegro')]
[('obj', 'http://www.wikidata.org/entity/Q237'), ('name', 'Vatican City')]
[('obj', 'http://www.wikidata.org/entity/Q238'), ('name', 'San Marino')]
[('obj', 'http://www.wikidata.org/entity/Q29'), ('name', 'Spain')]
[('obj', 'http://www.wikidata.org/entity/Q38'), ('name', 'Italy')]
[('obj', 'http://www.wikidata.org/entity/Q403'), ('n

In [183]:
# Looking at countries located in Northern Europe

queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q27479 ?p ?obj.
   # get the label
   ?obj sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q145'), ('name', 'United Kingdom')]
[('obj', 'http://www.wikidata.org/entity/Q189'), ('name', 'Iceland')]
[('obj', 'http://www.wikidata.org/entity/Q191'), ('name', 'Estonia')]
[('obj', 'http://www.wikidata.org/entity/Q20'), ('name', 'Norway')]
[('obj', 'http://www.wikidata.org/entity/Q211'), ('name', 'Latvia')]
[('obj', 'http://www.wikidata.org/entity/Q25230'), ('name', 'Guernsey')]
[('obj', 'http://www.wikidata.org/entity/Q33'), ('name', 'Finland')]
[('obj', 'http://www.wikidata.org/entity/Q34'), ('name', 'Sweden')]
[('obj', 'http://www.wikidata.org/entity/Q35'), ('name', 'Denmark')]
[('obj', 'http://www.wikidata.org/entity/Q37'), ('name', 'Lithuania')]
[('obj', 'http://www.wikidata.org/entity/Q46'), ('name', 'Europe')]
[('obj', 'http://www.wikidata.org/entity/Q4628'), ('name', 'Faroe Islands')]
[('obj', 'http://www.wikidata.org/entity/Q785'), ('name', 'Jersey')]
[('obj', 'http://www.wikidata.org/entity/Q82794'), ('name', 'geographic re

In [184]:
# Looking at countries located in Southeast Europe

queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q263686 ?p ?obj.
   # get the label
   ?obj sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q10815257'), ('name', 'Portal:Southeastern Europe')]
[('obj', 'http://www.wikidata.org/entity/Q84076816'), ('name', 'Category:Maps of Southeastern Europe')]
[('obj', 'http://www.wikidata.org/entity/Q9811465'), ('name', 'Category:Southeastern Europe')]
[('obj', 'http://www.wikidata.org/entity/Q46'), ('name', 'Europe')]
[('obj', 'http://www.wikidata.org/entity/Q82794'), ('name', 'geographic region')]
5


In [186]:
# Looking at countries located in Southwestern Europe

queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q50807777 ?p ?obj.
   # get the label
   ?obj sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q46'), ('name', 'Europe')]
[('obj', 'http://www.wikidata.org/entity/Q82794'), ('name', 'geographic region')]
2


In [187]:
# Looking at countries located in North Eastern Europe

queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q1998497 ?p ?obj.
   # get the label
   ?obj sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q27468'), ('name', 'Eastern Europe')]
[('obj', 'http://www.wikidata.org/entity/Q46'), ('name', 'Europe')]
[('obj', 'http://www.wikidata.org/entity/Q82794'), ('name', 'geographic region')]
3


In [188]:
# Looking at countries located in Northwestern Europe

queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q1414323 ?p ?obj.
   # get the label
   ?obj sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q46'), ('name', 'Europe')]
[('obj', 'http://www.wikidata.org/entity/Q82794'), ('name', 'geographic region')]
2


In [189]:
# Looking at countries located in Eurasia

queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q5401 ?p ?obj.
   # get the label
   ?obj sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q1324222'), ('name', 'Eurasianism')]
[('obj', 'http://www.wikidata.org/entity/Q15'), ('name', 'Africa')]
[('obj', 'http://www.wikidata.org/entity/Q19180675'), ('name', 'Small Brockhaus and Efron Encyclopedic Dictionary')]
[('obj', 'http://www.wikidata.org/entity/Q20078554'), ('name', 'Great Soviet Encyclopedia (1926–1947)')]
[('obj', 'http://www.wikidata.org/entity/Q46'), ('name', 'Europe')]
[('obj', 'http://www.wikidata.org/entity/Q48'), ('name', 'Asia')]
[('obj', 'http://www.wikidata.org/entity/Q49'), ('name', 'North America')]
[('obj', 'http://www.wikidata.org/entity/Q513'), ('name', 'Mount Everest')]
[('obj', 'http://www.wikidata.org/entity/Q5460604'), ('name', 'Wikipedia:Vital articles')]
[('obj', 'http://www.wikidata.org/entity/Q602358'), ('name', 'Brockhaus and Efron Encyclopedic Dictionary')]
[('obj', 'http://www.wikidata.org/entity/Q82794'), ('name', 'geographic region')]
[('obj', 'http://www.wikidata.org/entity/Q3960'), ('name'

In [ ]:
# There aren't useful European countries in Southeast Europe, Southwestern Europe, North Eastern Europe, Northwestern Europe, Eurasia

In [ ]:
# this method isn't really efficient because, if a new country is inserted in the Europe, the programmer will have to add manually its code in the values for countries
# looking at Europe properties it seems really strange that there's no way of getting all of its countries but also checking on the Wikidata server the only method is to use Europe subregions.

In [ ]:
# if instead it's the marathon the data added to the database, the query will update its result, if the marathon has the country property

In [ ]:
# Q184 Belarus, Q212 Ukraine, Q213 Czech Republic, Q214 Slovakia, Q217 Moldova, Q218 Romania, Q219 Bulgaria, Q28 Hungary, Q36 Poland, Q142 France, Q183 Germany, Q235 Monaco, Q31 Belgium, 
# Q32 Luxembourg, Q39 Switzerland, Q40 Austria, Q55 Netherlands, Q145 United Kingdom, Q27 Ireland, Q347 Liechtenstein, Q35 Denmark, Q1410 Gibraltar,
# Q215 Slovenia, Q221 North Macedonia, Q222 Albania, Q224 Croatia, Q225 Bosnia and Herzegovina, Q228 Andorra, Q233 Malta, Q236 Montenegro, Q237 Vatican City, Q238 San Marino,
# Q29 Spain, Q38 Italy, Q403 Serbia, Q41 Greece, Q45 Portugal, Q189 Iceland, Q191 Estonia, Q20 Norway, Q211 Latvia, Q25230 Guernsey, Q33 Finland, Q34 Sweden, Q37 Lithuania, Q4628 Faroe Islands,
# Q785 Jersey, Q9676 Isle of Man, Q203942 Galdhøpiggen, Q3405693 Sark

In [210]:
# let's now get all the marathons that took place in Eurore

queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   VALUES ?country {wd:Q183 wd:Q184 wd:Q212 wd:Q213 wd:Q214 wd:Q217 wd:Q218 wd:Q219 wd:Q28 wd:Q36 wd:Q142 wd:Q183 wd:Q235 wd:Q31 wd:Q32 wd:Q39 wd:Q40 wd:Q55 wd:Q145 wd:Q27 wd:Q347 wd:Q35 wd:Q1410
                    wd:Q215 wd:Q221 wd:Q222 wd:Q224 wd:Q225 wd:Q228 wd:Q233 wd:Q236 wd:Q237 wd:Q238 wd:Q29 wd:Q38 wd:Q403 wd:Q41 wd:Q45 wd:Q189 wd:Q191 wd:Q20 wd:Q211 wd:Q25230 wd:Q33 wd:Q34 wd:Q37
                    wd:Q4628}
   ?obj wdt:P279 wd:Q40244;
        wdt:P17 ?country.
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q161222'), ('name', 'Berlin Marathon')]
[('obj', 'http://www.wikidata.org/entity/Q528634'), ('name', 'Frankfurt Marathon')]
[('obj', 'http://www.wikidata.org/entity/Q1071285'), ('name', 'Paris Marathon')]
[('obj', 'http://www.wikidata.org/entity/Q679106'), ('name', 'Madrid Marathon')]
[('obj', 'http://www.wikidata.org/entity/Q807910'), ('name', 'Barcelona Marathon')]
[('obj', 'http://www.wikidata.org/entity/Q1980764'), ('name', 'Neujahrsmarathon Zürich')]
[('obj', 'http://www.wikidata.org/entity/Q578794'), ('name', 'London Marathon')]
[('obj', 'http://www.wikidata.org/entity/Q837910'), ('name', 'Turin Marathon')]
[('obj', 'http://www.wikidata.org/entity/Q1258425'), ('name', 'Dresden Marathon')]
[('obj', 'http://www.wikidata.org/entity/Q748757'), ('name', 'Rome Marathon')]
[('obj', 'http://www.wikidata.org/entity/Q3287504'), ('name', 'Marathon du beaujolais')]
[('obj', 'http://www.wikidata.org/entity/Q1477977'), ('name', 'Stockholm Marath

In [ ]:
# we had discovered the winner property for the 2008 Summer Olympics – men's 200 metres: wdt:P1346 -- winner
# it's not assured that the property is available also for marathons event. Let's check that

In [218]:
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q161222 ?p ?runner.
   # get the label
   ?p sc:name ?name.
}
#LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('name', 'located in the administrative territorial entity')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P2002'), ('name', 'Twitter username')]
[('p', 'http://www.wikidata.org/prop/direct/P2031'), ('name', 'work period (start)')]
[('p', 'http://www.wikidata.org/prop/direct/P214'), ('name', 'VIAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('name', 'location')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P2922'), ('name', 'month of the year')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'htt

In [ ]:
# we still can't find a way to get marathon winners

In [ ]:
# now that we have all marathons that took place in Europe we can proceed to the final query

Final query for this task

In [213]:
# Consider marathons ran in Europe. Return the number of different runners who won a marathon group by their country of citizenship 
# (the result set must be a triples of country IRI, label and #different runners)
# ???? -- marathon winner
# wdt:P279 -- subclass of 
# wdt:Q40244 -- marathon
# wdt:P17 -- country

# wdt:P27 -- country of citizenship

queryString = """
SELECT DISTINCT ?country ?name (COUNT(?citizenship) AS ?differentRunners)
WHERE {
   # bind something
   VALUES ?country {wd:Q183 wd:Q184 wd:Q212 wd:Q213 wd:Q214 wd:Q217 wd:Q218 wd:Q219 wd:Q28 wd:Q36 wd:Q142 wd:Q183 wd:Q235 wd:Q31 wd:Q32 wd:Q39 wd:Q40 wd:Q55 wd:Q145 wd:Q27 wd:Q347 wd:Q35 wd:Q1410
                    wd:Q215 wd:Q221 wd:Q222 wd:Q224 wd:Q225 wd:Q228 wd:Q233 wd:Q236 wd:Q237 wd:Q238 wd:Q29 wd:Q38 wd:Q403 wd:Q41 wd:Q45 wd:Q189 wd:Q191 wd:Q20 wd:Q211 wd:Q25230 wd:Q33 wd:Q34 wd:Q37
                    wd:Q4628}
   ?obj wdt:P279 wd:Q40244;
        wdt:P17 ?country.
   ?obj (marathon winner) ?runner.
   ?runner wdt:P27 ?citizenship.
   # get the label
   ?country sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
Empty


## Task 5
Identify the BGP for Olympic Games

In [36]:
# let's analize an evenet of olimpic games 

queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q1649415 wdt:P31 ?obj.
   # get the label
   ?obj sc:name ?name.
}
#LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q18536594'), ('name', 'Olympic sporting event')]
[('obj', 'http://www.wikidata.org/entity/Q11783626'), ('name', 'athletics meeting')]
2


In [32]:
# let's get entities related to 'Olympic sporting event'

queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q18536594 ?p ?obj.
   # get the label
   ?obj sc:name ?name.
}
#LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q16510064'), ('name', 'sporting event')]
[('obj', 'http://www.wikidata.org/entity/Q212434'), ('name', 'Olympic sport')]
[('obj', 'http://www.wikidata.org/entity/Q26132862'), ('name', 'Olympic sports discipline event')]
[('obj', 'http://www.wikidata.org/entity/Q50393057'), ('name', 'Paralympic sporting event')]
[('obj', 'http://www.wikidata.org/entity/Q84491920'), ('name', 'Youth Olympic sporting event')]
[('obj', 'http://www.wikidata.org/entity/Q5852808'), ('name', 'Template:Infobox Olympic event')]
6


Final query for this task

In [53]:
# let's get entities related to Olympic sport'

queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q212434 ?p ?obj.
   # get the label
   ?obj sc:name ?name.
}
#LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q51031626'), ('name', 'sport competition at a multi-sport event')]
[('obj', 'http://www.wikidata.org/entity/Q18536594'), ('name', 'Olympic sporting event')]
[('obj', 'http://www.wikidata.org/entity/Q26132862'), ('name', 'Olympic sports discipline event')]
[('obj', 'http://www.wikidata.org/entity/Q31629'), ('name', 'type of sport')]
[('obj', 'http://www.wikidata.org/entity/Q349'), ('name', 'sport')]
[('obj', 'http://www.wikidata.org/entity/Q5389'), ('name', 'Olympic Games')]
[('obj', 'http://www.wikidata.org/entity/Q589184'), ('name', 'paralympic sports')]
[('obj', 'http://www.wikidata.org/entity/Q58825429'), ('name', 'Olympic competitor')]
[('obj', 'http://www.wikidata.org/entity/Q7024248'), ('name', 'Category:Olympic sports')]
[('obj', 'http://www.wikidata.org/entity/Q5917548'), ('name', 'Template:Sports at the Olympics')]
10


In [ ]:
#[('obj', 'http://www.wikidata.org/entity/Q5389'), ('name', 'Olympic Games')]

## Task 6
Consider only the summer olympic games of 2004-2008-2012-2016. Return the number of marathon runners who had partecipated to all these editions group by their country of citizenship (if the runner partecipated in more than one edition of the game, consider him/her only once) (the result set must be a list of country IRI, label and #marathon runners)

In [ ]:
# let's start getting the entities for summer olympic games of 2004-2008-2012-2016

In [ ]:
# from a previous query : [('obj', 'http://www.wikidata.org/entity/Q1649415'), ('name', "athletics at the 2012 Summer Olympics – men's 200 metres")]

In [63]:
# let's get entities related to 'athletics at the 2012 Summer Olympics – men's 200 metres'

queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q1649415 ?p ?obj.
   # get the label
   ?obj sc:name ?name.
}
#LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q172190'), ('name', 'London Stadium')]
[('obj', 'http://www.wikidata.org/entity/Q145'), ('name', 'United Kingdom')]
[('obj', 'http://www.wikidata.org/entity/Q185262'), ('name', 'athletics at the 2012 Summer Olympics')]
[('obj', 'http://www.wikidata.org/entity/Q18536594'), ('name', 'Olympic sporting event')]
[('obj', 'http://www.wikidata.org/entity/Q40970'), ('name', 'International Olympic Committee')]
[('obj', 'http://www.wikidata.org/entity/Q542'), ('name', 'athletics')]
[('obj', 'http://www.wikidata.org/entity/Q11783626'), ('name', 'athletics meeting')]
[('obj', 'http://www.wikidata.org/entity/Q1189'), ('name', 'Usain Bolt')]
[('obj', 'http://www.wikidata.org/entity/Q55243755'), ('name', "men's 200 metres")]
[('obj', 'http://www.wikidata.org/entity/Q853003'), ('name', "athletics at the 2008 Summer Olympics – men's 200 metres")]
[('obj', 'http://www.wikidata.org/entity/Q26219856'), ('name', "athletics at the 2016 Summer Olympics – men's

In [71]:
# let's use the 'instance of' (wdt:P31) with 'athletics at the 2012 Summer Olympics' (wd:Q185262)

queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q185262 wdt:P31 ?obj.
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q26132862'), ('name', 'Olympic sports discipline event')]
[('obj', 'http://www.wikidata.org/entity/Q715044'), ('name', 'athletics at the Summer Olympics')]
2


In [73]:
# let's get entities related to 'athletics at the Summer Olympics'

queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q715044 ?p ?obj.
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q159821'), ('name', 'Summer Olympic Games')]
[('obj', 'http://www.wikidata.org/entity/Q18608583'), ('name', 'recurring sporting event')]
[('obj', 'http://www.wikidata.org/entity/Q26132862'), ('name', 'Olympic sports discipline event')]
[('obj', 'http://www.wikidata.org/entity/Q40970'), ('name', 'International Olympic Committee')]
[('obj', 'http://www.wikidata.org/entity/Q542'), ('name', 'athletics')]
[('obj', 'http://www.wikidata.org/entity/Q8274356'), ('name', 'Category:Athletics at the Summer Olympics')]
6


In [81]:
# let's get entities related to 'Summer Olympic Games'

queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q159821 ?p ?obj.
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q8098'), ('name', '1904 Summer Olympics')]
[('obj', 'http://www.wikidata.org/entity/Q8150'), ('name', '1936 Summer Olympics')]
[('obj', 'http://www.wikidata.org/entity/Q8415'), ('name', '1960 Summer Olympics')]
[('obj', 'http://www.wikidata.org/entity/Q181278'), ('name', '2020 Summer Olympics')]
[('obj', 'http://www.wikidata.org/entity/Q18608583'), ('name', 'recurring sporting event')]
[('obj', 'http://www.wikidata.org/entity/Q40970'), ('name', 'International Olympic Committee')]
[('obj', 'http://www.wikidata.org/entity/Q5389'), ('name', 'Olympic Games')]
[('obj', 'http://www.wikidata.org/entity/Q8088'), ('name', '1900 Summer Olympics')]
[('obj', 'http://www.wikidata.org/entity/Q82414'), ('name', 'Winter Olympic Games')]
[('obj', 'http://www.wikidata.org/entity/Q8438'), ('name', '1972 Summer Olympics')]
[('obj', 'http://www.wikidata.org/entity/Q8444'), ('name', '1976 Summer Olympics')]
[('obj', 'http://www.wikidata.org/entity/Q8470'), ('

In [75]:
# we finally get the entities wanted

#[('obj', 'http://www.wikidata.org/entity/Q8558'), ('name', '2004 Summer Olympics')]
#[('obj', 'http://www.wikidata.org/entity/Q8567'), ('name', '2008 Summer Olympics')]
#[('obj', 'http://www.wikidata.org/entity/Q8577'), ('name', '2012 Summer Olympics')]
#[('obj', 'http://www.wikidata.org/entity/Q8613'), ('name', '2016 Summer Olympics')]

In [82]:
# let's get entities related to '2004 Summer Olympics'

queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q8558 ?p ?obj.
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q118446'), ('name', 'rowing at the 2004 Summer Olympics')]
[('obj', 'http://www.wikidata.org/entity/Q1347166'), ('name', 'Lazaros Voreadis')]
[('obj', 'http://www.wikidata.org/entity/Q141340'), ('name', 'Kiribati at the 2004 Summer Olympics')]
[('obj', 'http://www.wikidata.org/entity/Q1524'), ('name', 'Athens')]
[('obj', 'http://www.wikidata.org/entity/Q15637418'), ('name', 'artistic gymnastics at the 2004 Summer Olympics')]
[('obj', 'http://www.wikidata.org/entity/Q159821'), ('name', 'Summer Olympic Games')]
[('obj', 'http://www.wikidata.org/entity/Q1602350'), ('name', 'rhythmic gymnastics at the 2004 Summer Olympics')]
[('obj', 'http://www.wikidata.org/entity/Q18536594'), ('name', 'Olympic sporting event')]
[('obj', 'http://www.wikidata.org/entity/Q189663'), ('name', 'Olympic Stadium of Athens')]
[('obj', 'http://www.wikidata.org/entity/Q2022374'), ('name', 'trampoline gymnastics at the 2004 Summer Olympics')]
[('obj', 'http://www.wiki

In [83]:
# get all the marathon runners
queryString = """
SELECT DISTINCT ?runner ?name
WHERE {
   # bind something
   ?runner wdt:P106 wd:Q13382460;
        wdt:P31 wd:Q5.
   # get the label
   ?runner sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('runner', 'http://www.wikidata.org/entity/Q1701191'), ('name', 'John Nada Saya')]
[('runner', 'http://www.wikidata.org/entity/Q16006623'), ('name', 'Harry Payne')]
[('runner', 'http://www.wikidata.org/entity/Q16007889'), ('name', 'John Tarrant')]
[('runner', 'http://www.wikidata.org/entity/Q16008400'), ('name', 'Waldemar Karlsson')]
[('runner', 'http://www.wikidata.org/entity/Q16011969'), ('name', 'John J. Reynolds')]
[('runner', 'http://www.wikidata.org/entity/Q1602744'), ('name', 'Hellen Mugo')]
[('runner', 'http://www.wikidata.org/entity/Q16185418'), ('name', 'Jerry Kiernan')]
[('runner', 'http://www.wikidata.org/entity/Q16186777'), ('name', 'George Malley')]
[('runner', 'http://www.wikidata.org/entity/Q16210841'), ('name', 'Elva Dryer')]
[('runner', 'http://www.wikidata.org/entity/Q16212704'), ('name', 'José Alejandro Semprún')]
10


In [ ]:
# let's look at properties of a specific runner to see if we get a property that says "attended in"

queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q1701191 ?p ?runner.
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P1146'), ('name', 'World Athletics athlete ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('name', 'languages spoken, written or signed')]
[('p', 'http://www.wikidata.org/prop/direct/P1447'), ('name', 'Sports-Reference.com Olympic athlete ID')]
[('p', 'http://www.wikidata.org/prop/direct/P19'), ('name', 'place of birth')]
[('p', 'http://www.wikidata.org/prop/direct/P21'), ('name', 'sex or gender')]
[('p', 'http://www.wikidata.org/prop/direct/P2416'), ('name', 'sports discipline competed in')]
[('p', 'http://www.wikidata.org/prop/direct/P27'), ('name', 'country of citizenship')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/

In [ ]:
# [('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]

In [ ]:
# our goal is to return the number of marathon runners who had partecipated to all these editions group by their country of citizenship

In [93]:
# let's first try to do it for the 2004 edition

queryString = """
SELECT DISTINCT ?country ?name (COUNT (?country) AS ?countries)
WHERE {
   # bind something
   ?runner1 wdt:P106 wd:Q13382460;
        wdt:P1344 wd:Q8558;
        #wdt:P1344 wd:Q8567;
        #wdt:P1344 wd:Q8577;
        #wdt:P1344 wd:Q8613;
        wdt:P31 wd:Q5;
        wdt:P27 ?country.
   # get the label
   ?country sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q1036'), ('name', 'Uganda'), ('countries', '1')]
[('country', 'http://www.wikidata.org/entity/Q398'), ('name', 'Bahrain'), ('countries', '1')]
[('country', 'http://www.wikidata.org/entity/Q191'), ('name', 'Estonia'), ('countries', '2')]
[('country', 'http://www.wikidata.org/entity/Q159'), ('name', 'Russia'), ('countries', '7')]
[('country', 'http://www.wikidata.org/entity/Q45'), ('name', 'Portugal'), ('countries', '4')]
[('country', 'http://www.wikidata.org/entity/Q664'), ('name', 'New Zealand'), ('countries', '1')]
[('country', 'http://www.wikidata.org/entity/Q865'), ('name', 'Taiwan'), ('countries', '1')]
[('country', 'http://www.wikidata.org/entity/Q739'), ('name', 'Colombia'), ('countries', '2')]
[('country', 'http://www.wikidata.org/entity/Q211'), ('name', 'Latvia'), ('countries', '1')]
[('country', 'http://www.wikidata.org/entity/Q717'), ('name', 'Venezuela'), ('countries', '1')]
[('country', 'http://www.wikidata.org/entity/Q39

In [134]:
#test

queryString = """
SELECT DISTINCT ?country ?name ?event ?eventName (COUNT (?country) AS ?countries)
WHERE {
   # bind something
   VALUES ?runner1 {wd:Q9124}.
   ?runner1 wdt:P106 wd:Q13382460;
        wdt:P1344 ?event;
        #wdt:P1344 {wd:Q8558, wd:Q8567};
        #wdt:P1344 wd:Q8567;
        #wdt:P1344 wd:Q8577;
        #wdt:P1344 wd:Q8613;
        wdt:P31 wd:Q5;
        wdt:P27 ?country.
   # get the label
   ?country sc:name ?name.
   ?event sc:name ?eventName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
The operation failed timed out


Final query for this task

In [133]:
#Return the number of marathon runners who had partecipated to all these editions (2004-2008-2012-2016) group by their country of citizenship 
#(if the runner partecipated in more than one edition of the game, consider him/her only once) (the result set must be a list of country IRI, label and #marathon runners)

queryString = """
SELECT DISTINCT ?country ?name (COUNT (?country) AS ?marathonRunners)
WHERE {
   # bind something
   VALUES ?event {wd:Q8558 wd:PQ8567 wd:Q8577 wd:Q8613}.
   ?runner1 wdt:P106 wd:Q13382460;
        wdt:P1344 ?event;
        #wdt:P1344 {wd:Q8558, wd:Q8567};
        #wdt:P1344 wd:Q8567;
        #wdt:P1344 wd:Q8577;
        #wdt:P1344 wd:Q8613;
        wdt:P31 wd:Q5;
        wdt:P27 ?country.
   # get the label
   ?country sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q1036'), ('name', 'Uganda'), ('marathonRunners', '4')]
[('country', 'http://www.wikidata.org/entity/Q191'), ('name', 'Estonia'), ('marathonRunners', '2')]
[('country', 'http://www.wikidata.org/entity/Q398'), ('name', 'Bahrain'), ('marathonRunners', '2')]
[('country', 'http://www.wikidata.org/entity/Q668'), ('name', 'India'), ('marathonRunners', '1')]
[('country', 'http://www.wikidata.org/entity/Q189'), ('name', 'Iceland'), ('marathonRunners', '1')]
[('country', 'http://www.wikidata.org/entity/Q159'), ('name', 'Russia'), ('marathonRunners', '13')]
[('country', 'http://www.wikidata.org/entity/Q211'), ('name', 'Latvia'), ('marathonRunners', '2')]
[('country', 'http://www.wikidata.org/entity/Q45'), ('name', 'Portugal'), ('marathonRunners', '9')]
[('country', 'http://www.wikidata.org/entity/Q664'), ('name', 'New Zealand'), ('marathonRunners', '2')]
[('country', 'http://www.wikidata.org/entity/Q717'), ('name', 'Venezuela'), ('marathonRunne

## Task 7
For each disciplines of the long distance running find who hold the World Record (the result set must be a list of elements with discipline IRI, label and athlete IRI and label).

In [ ]:
# look at entities that have'long-distance running' as a subclass
# wdt:P279 -- subclass
# wd:Q917206 -- 'long-distance running'

queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   ?obj wdt:P279 wd:Q917206.
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

In [39]:
# let's find out the property for 'record held'

queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q40244 ?obj ?p.
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/prop/direct/P1000'), ('name', 'record held')]
[('obj', 'http://www.wikidata.org/prop/direct/P1225'), ('name', 'U.S. National Archives Identifier')]
[('obj', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('obj', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('obj', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('obj', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('obj', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('obj', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('obj', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('obj', 'http://www.wikidata.org/prop/direct/P2347'), ('name', 'YSO ID')]
[('obj', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('obj', 'http://www.wikidat

Final query for this task

In [77]:
# find out who hold the World Record for each disciplines of the long distance running

# wdt:P279 -- subclass
# wd:Q917206 -- 'long-distance running'
# wdt:P1000 -- record held

queryString = """
SELECT DISTINCT ?discipline ?name ?runner ?fullName
WHERE {
   # bind something
   ?discipline wdt:P279 wd:Q917206.
   ?runner wdt:P1000 ?discipline.
   # get the label
   ?discipline sc:name ?name.
   ?runner sc:name ?fullName.
   
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('discipline', 'http://www.wikidata.org/entity/Q163892'), ('name', '10,000 metres'), ('runner', 'http://www.wikidata.org/entity/Q9009157'), ('fullName', 'Inés Melchor')]
[('discipline', 'http://www.wikidata.org/entity/Q163892'), ('name', '10,000 metres'), ('runner', 'http://www.wikidata.org/entity/Q5930934'), ('fullName', 'Jhon Cusi Huaman')]
[('discipline', 'http://www.wikidata.org/entity/Q240500'), ('name', '5000 metres'), ('runner', 'http://www.wikidata.org/entity/Q98780335'), ('fullName', 'Mike Foppen')]
[('discipline', 'http://www.wikidata.org/entity/Q240500'), ('name', '5000 metres'), ('runner', 'http://www.wikidata.org/entity/Q9119'), ('fullName', 'Kenenisa Bekele Beyecha')]
[('discipline', 'http://www.wikidata.org/entity/Q2164200'), ('name', 'one hour run'), ('runner', 'http://www.wikidata.org/entity/Q171500'), ('fullName', 'Gebrselassie Haile')]
5


In [ ]:
# due to the lack of datas in the snapshot the query doesn't show the record of each discipline.